### Imports

In [ ]:
import fiona
import shapely.geometry
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

### Create basemap for plotting in matplotlib

In [ ]:
ny_map = Basemap(projection='merc', lat_0=40.6, lon_0=-73.9, llcrnrlat=40.5,urcrnrlat=40.9, llcrnrlon=-74.3,urcrnrlon=-73.65,resolution='h', area_thresh=1)

### Create income table from census data and merge with the GEO_ID to match polygons with income

In [ ]:
income_table = pd.read_csv('/Users/murdock/Documents/metis/MTABenson_metis/ACS_15_5YR_B19013_with_ann.csv')

In [ ]:
income_table = income_table.drop(income_table.index[0])
income_table.columns=['GEO_ID', 'GEO_ID2', 'GEO_DISPLAY_LABEL', 'MEDIAN_INCOME', 'MARGIN_OF_ERROR']
income_table.head()

In [ ]:
filename = '/Users/murdock/Documents/metis/MTABenson_metis/thematic_map_shape/nyshapefile.shp'
blocks = []
with fiona.open(filename) as data:
    for block_data in data:
        geometry = shapely.geometry.shape(block_data['geometry'])
        block = block_data['properties']['GEO_ID']
        blocks.append([block, block_data['geometry']['coordinates'][0]])

In [ ]:
df = pd.DataFrame(blocks, columns=['GEO_ID', 'COORDINATES'])
df.head()

In [ ]:
map_df = pd.merge(df, income_table, on='GEO_ID', how='left')
map_df.head()

In [ ]:
map_df['MEDIAN_INCOME'] = map_df['MEDIAN_INCOME'].astype(int)

### Create figure and color code by normalizing the income for each shape

In [ ]:
fig = plt.figure(figsize=(16, 16))
ax = fig.add_subplot(111)
ny_map.drawmapboundary(fill_color='aqua')
ny_map.fillcontinents(color='#f2f2f2',lake_color='aqua')
ny_map.drawcoastlines()

ny_map.readshapefile('/Users/murdock/Documents/metis/MTABenson_metis/thematic_map_shape/nyshapefile', 'nyshapefile')

cmap = plt.get_cmap('Oranges')

patches   = []

for info, shape in zip(ny_map.nyshapefile_info, ny_map.nyshapefile):
    patches.append(Polygon(np.array(shape), True))

norm = Normalize()
pc = PatchCollection(patches, zorder=2)
 
#pc.set_facecolor(cmap(norm(mapping_df['MEDIAN_INCOME'].fillna(0).values)))
pc.set_facecolor(cmap(norm(map_df['MEDIAN_INCOME'].fillna(0).values)))
ax.add_collection(pc)

mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)

mapper.set_array(map_df['MEDIAN_INCOME'])
plt.colorbar(mapper, shrink=0.4)

plt.show()
#plt.savefig('NY_incomemap.png', bbox_inches='tight')

### Load table of selected train stations and plot these points on the income map

In [ ]:
station_path = '/Users/murdock/Documents/Metis/MTABenson_metis/pklfiles/'
final_stations_df = pd.read_pickle(station_path + 'final_stations.pkl')

In [ ]:
final_stations_df.head()

### Plotting map with top stations overlaid as points on the map

In [ ]:
ny_map_with_stations = Basemap(projection='merc', lat_0=40.6, lon_0=-73.9, llcrnrlat=40.67,urcrnrlat=40.83, llcrnrlon=-74.08,urcrnrlon=-73.9,resolution='h', area_thresh=1)

In [ ]:
fig = plt.figure(figsize=(16, 16))
ax = fig.add_subplot(111)
ny_map_with_stations.drawmapboundary(fill_color='aqua')
ny_map_with_stations.fillcontinents(color='#f2f2f2',lake_color='aqua')
ny_map_with_stations.drawcoastlines()

ny_map_with_stations.readshapefile('/Users/murdock/Documents/metis/MTABenson_metis/thematic_map_shape/nyshapefile', 'nyshapefile')

cmap = plt.get_cmap('Oranges')

patches   = []

for info, shape in zip(ny_map_with_stations.nyshapefile_info, ny_map_with_stations.nyshapefile):
    patches.append(Polygon(np.array(shape), True))

norm = Normalize()
pc = PatchCollection(patches, zorder=2)

pc.set_facecolor(cmap(norm(map_df['MEDIAN_INCOME'].fillna(0).values)))
ax.add_collection(pc)

lons = final_stations_df['LONGITUDE'].values
lats = final_stations_df['LATITUDE'].values
x, y = ny_map_with_stations(lons, lats)
ny_map_with_stations.plot(x, y, 'bo', markersize=20)

labels = [name for name in final_stations_df['STATION'].values]
for label, xpt, ypt in zip(labels, x, y):
    plt.text(xpt, ypt, label)

mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)

mapper.set_array(map_df['MEDIAN_INCOME'])
plt.colorbar(mapper, shrink=0.4)

plt.show()
#plt.savefig('NY_incomemap_plus_stations.png', bbox_inches='tight')